In [23]:
import torch
import numpy as np
import pandas as pd

from train_model import run_model, MODELS, WordModel, train_model, test_model
from data_processor import DoqProcessor, VerbProcessor, ProstProcessor

In [2]:
doq_train = DoqProcessor('train')
train_labels, _, train_sentences, _ = doq_train.forward()

doq_test = DoqProcessor('test')
test_labels, _, test_sentences, _ = doq_test.forward()

doq = {}
doq['train'] = {
    'labels': train_labels,
    'sents': train_sentences,
}

doq['test'] = {
    'labels': test_labels,
    'sents': test_sentences,
}

In [3]:
prost_train = ProstProcessor('train', batch_size=0)
train_labels, _, train_sentences, _ = prost_train.forward()

prost_test = ProstProcessor('test', batch_size=0)
test_labels, _, test_sentences, _ = prost_test.forward()

prost = {}
prost['train'] = {
    'labels': train_labels,
    'sents': train_sentences,
}

prost['test'] = {
    'labels': test_labels,
    'sents': test_sentences,
}

Using custom data configuration default
Reusing dataset prost (/Users/g/.cache/huggingface/datasets/corypaik___prost/default/1.0.1/5723fec956da2bd5e34a0ac6f53f398b771fbb743fc979c6a50b513888d537c7)
Loading cached split indices for dataset at /Users/g/.cache/huggingface/datasets/corypaik___prost/default/1.0.1/5723fec956da2bd5e34a0ac6f53f398b771fbb743fc979c6a50b513888d537c7/cache-3ba89f75d16d1d23.arrow and /Users/g/.cache/huggingface/datasets/corypaik___prost/default/1.0.1/5723fec956da2bd5e34a0ac6f53f398b771fbb743fc979c6a50b513888d537c7/cache-dea55751f6b71d2b.arrow
Using custom data configuration default
Reusing dataset prost (/Users/g/.cache/huggingface/datasets/corypaik___prost/default/1.0.1/5723fec956da2bd5e34a0ac6f53f398b771fbb743fc979c6a50b513888d537c7)
Loading cached split indices for dataset at /Users/g/.cache/huggingface/datasets/corypaik___prost/default/1.0.1/5723fec956da2bd5e34a0ac6f53f398b771fbb743fc979c6a50b513888d537c7/cache-556ac204ac069f7c.arrow and /Users/g/.cache/huggingf

In [4]:
verb = {}
for attr in ['weight', 'speed', 'size', 'strength', 'rigidness']:
    verb_train = VerbProcessor(attr, 20, 'train', 0)
    train_labels, _, train_sentences, _ = verb_train.forward()

    verb_test = VerbProcessor(attr, 20, 'test', 0)
    test_labels, _, test_sentences, _ = verb_test.forward()

    verb[attr] = {}
    verb[attr]['train'] = {
        'labels': train_labels,
        'sents': train_sentences,
    }

    verb[attr]['test'] = {
        'labels': test_labels,
        'sents': test_sentences,
    }

In [6]:
def evaluate_prost(model):
    if model == 'roberta':
        return None
    
    m = WordModel(model, try_encoder=True)
    train_features = m.get_features(prost['train']['sents'])
    test_features = m.get_features(prost['test']['sents'])
    
    lr = train_model(train_features, prost['train']['labels'])
    acc, probs = test_model(lr, test_features, prost['test']['labels'], return_probs=True)
    
    assert len(probs) % 4 == 0
    
    correct_count = 0.0
    
    for i in range(0, probs.shape[0], 4):
        example = probs[i:i+4]
        true_pred = np.max(example, axis=0)
        
        for j, ex in enumerate(example):
            if ex[1] == true_pred[1] and prost['test']['labels'][i + j]:
                correct_count += 1
                break
    
    return correct_count / (probs.shape[0] / 4) * 100

In [ ]:
# Non-pooled evaluation
for model in ['visualbert']:
    print(model)
    doq_acc = None
    prost_acc = None
    try:
        # doq_acc = run_model(model, doq['train']['labels'], doq['train']['sents'], doq['test']['labels'], doq['test']['sents'], try_encoder=True, verbose=False)
        # print("doq", doq_acc)
        # verb_acc = {}
        # for attr in verb:
        #     verb_acc[attr] = run_model(model, verb[attr]['train']['labels'], verb[attr]['train']['sents'], verb[attr]['test']['labels'], verb[attr]['test']['sents'], try_encoder=True, verbose=False)
        #     print(f"verb-{attr}", verb_acc[attr])
        prost_acc = evaluate_prost(model)
        print("prost", prost_acc)
    except Exception as e:
        print(e)
    
    # model_accs[model] = {
    #     'doq': doq_acc,
    #     'prost': prost_acc,
    # }
    
    # for attr in verb:
    #     model_accs[model][f'verb_{attr}'] = verb_acc[attr]
    model_accs[model]['prost'] = prost_acc

visualbert


In [ ]:
model_accs

In [ ]:
# Roberta fails on prost! bummer Also visualbert??
prost_acc = evaluate_prost('roberta')

In [ ]:
prost_acc

In [14]:
df = pd.DataFrame(model_accs)

In [18]:
df.T

,doq,prost,verb_weight,verb_speed,verb_size,verb_strength,verb_rigidness
clip,66.727273,0.336179,78.510473,69.890511,79.209486,72.966102,67.799490
roberta,64.000000,NaN,84.173778,75.273723,82.608696,78.559322,71.452846
roberta_small,59.545455,0.659552,71.683476,62.226277,72.569170,68.983051,64.401020
visualbert,62.818182,0.324973,80.837859,76.824818,83.873518,80.000000,74.341546


In [21]:
pf = df.mean(axis=1)

In [33]:
multimodal = df.clip + df.visualbert

TypeError: unsupported operand type(s) for +: 'method' and 'float'

In [50]:
pf = pf.T

In [54]:
multimodal = (pf['clip'] + pf['visualbert'])/2
unimodal = (pf['roberta'] + pf['roberta_small'])/2

In [55]:
multimodal

doq               64.772727
prost             33.057631
verb_weight       79.674166
verb_speed        73.357664
verb_size         81.541502
verb_strength     76.483051
verb_rigidness    71.070518
dtype: float64

In [58]:
unimodal

doq               61.772727
prost             65.955176
verb_weight       77.928627
verb_speed        68.750000
verb_size         77.588933
verb_strength     73.771186
verb_rigidness    67.926933
dtype: float64

In [69]:
models = pd.concat([multimodal, unimodal], axis=1)
models.columns =['visual+language', 'language']
models.T

,doq,prost,verb_weight,verb_speed,verb_size,verb_strength,verb_rigidness
visual+language,64.772727,33.057631,79.674166,73.357664,81.541502,76.483051,71.070518
language,61.772727,65.955176,77.928627,68.750000,77.588933,73.771186,67.926933


In [19]:
def evaluate_prost(model):
    m = WordModel(model, try_encoder=True)
    train_features = m.get_features(prost['train']['sents'])
    test_features = m.get_features(prost['test']['sents'])
    
    lr = train_model(train_features, prost['train']['labels'])
    acc, probs = test_model(lr, test_features, prost['test']['labels'], return_probs=True)
    
    assert len(probs) % 4 == 0
    
    correct_count = 0.0
    
    for i in range(0, probs.shape[0], 4):
        example = probs[i:i+4]
        true_pred = np.max(example, axis=0)
        
        for j, ex in enumerate(example):
            if ex[1] == true_pred[1] and prost['test']['labels'][i + j]:
                correct_count += 1
                break
    
    return correct_count / (probs.shape[0] / 4) * 100

In [6]:
for model in ['roberta_small', 'roberta', 't5', 'clip', 'visualbert', 'lxmert']:
    prost_acc = evaluate_prost(model)

roberta 0.925
roberta_small 0.79


AttributeError: 'VisualBertForQuestionAnswering' object has no attribute 'embeddings'

In [12]:
from sklearn.linear_model import LogisticRegression, SGDClassifier


In [14]:
?SGDClassifier.fit

Signature:
SGDClassifier.fit(
    self,
    X,
    y,
    coef_init=None,
    intercept_init=None,
    sample_weight=None,
)
Docstring:
Fit linear model with Stochastic Gradient Descent.

Parameters
----------
X : {array-like, sparse matrix}, shape (n_samples, n_features)
    Training data.

y : ndarray of shape (n_samples,)
    Target values.

coef_init : ndarray of shape (n_classes, n_features), default=None
    The initial coefficients to warm-start the optimization.

intercept_init : ndarray of shape (n_classes,), default=None
    The initial intercept to warm-start the optimization.

sample_weight : array-like, shape (n_samples,), default=None
    Weights applied to individual samples.
    If not provided, uniform weights are assumed. These weights will
    be multiplied with class_weight (passed through the
    constructor) if class_weight is specified.

Returns
-------
self : object
    Returns an instance of self.
File:      ~/Desktop/ClassesFall21/cs197/.env/lib/python3.9/site

In [20]:
accs_embed = {}

# Non-pooled evaluation
for model in ['clip', 'roberta_small']:
    print(model)
    doq_acc = None
    prost_acc = None
    try:
        doq_acc = run_model(model, doq['train']['labels'], doq['train']['sents'], doq['test']['labels'], doq['test']['sents'], try_encoder=False, verbose=False)
        print("doq", doq_acc)
        verb_acc = {}
        for attr in verb:
            verb_acc[attr] = run_model(model, verb[attr]['train']['labels'], verb[attr]['train']['sents'], verb[attr]['test']['labels'], verb[attr]['test']['sents'], try_encoder=False, verbose=False)
            print(f"verb-{attr}", verb_acc[attr])
        prost_acc = evaluate_prost(model)
        print("prost", prost_acc)
    except Exception as e:
        print(e)
    
    accs_embed[model] = {
        'doq': doq_acc,
        'prost': prost_acc,
    }
    
    for attr in verb:
        accs_embed[model][f'verb_{attr}'] = verb_acc[attr]

clip
doq 63.36363636363637
verb-weight 80.99301784328937
verb-speed 70.71167883211679
verb-size 82.84584980237155
verb-strength 77.37288135593221
verb-rigidness 73.06711979609176
prost 91.94236926360726
roberta_small
doq 61.0
verb-weight 69.27851047323506
verb-speed 60.31021897810219
verb-size 69.56521739130434
verb-strength 64.40677966101696
verb-rigidness 61.34239592183518
prost 84.15154749199573


In [28]:
accs_df = accs_df.T

In [17]:
multimodal = (accs_df['clip'] + accs_df['lxmert'] + accs_df['visualbert'])/3
unimodal = (accs_df['roberta'] + accs_df['t5'] + accs_df['roberta_small'])/3

In [18]:
models = pd.concat([multimodal, unimodal], axis=1)
models.columns =['visual+language', 'language']
models.T

,doq,prost,verb_weight,verb_speed,verb_size,verb_strength,verb_rigidness
visual+language,63.484848,34.009249,81.872252,71.472019,82.29249,77.966102,71.651090
language,62.606061,49.501957,78.019136,68.248175,78.26087,73.389831,68.677428


In [27]:
from sklearn.decomposition import PCA

In [31]:
m = WordModel('roberta')

train_features = m.get_features(prost['train']['sents'])

In [34]:
train_features.shape

torch.Size([2000, 15360])